# SQL

Цель исследования - проанализировать базу данных о книгах, чтобы понять, что учесть в новом продукте.
<br>Для этого определить:
1. Какое количество книг в базе
2. Какие книги самые популярные
3. Какие издательства самые крупные
4. Какие авторы самые популярные
5. Как много книг читают самые активные пользователи

## Импорт библиотек

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 
import datetime as dt

## Функции

In [2]:
# функция для вывода запроса
def request(query):
    return pd.io.sql.read_sql(query, con = engine)

## Установка параметров

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

In [4]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']) 

In [5]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Обзор таблиц

In [6]:
query = '''SELECT *
           FROM 
        '''

### Таблица books

In [7]:
request(query+'books').head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

### Таблица authors

In [8]:
request(query+'authors').head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

### Таблица publishers

In [9]:
request(query+'publishers').head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

### Таблица ratings

In [10]:
request(query+'ratings').head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

### Таблица reviews

In [11]:
request(query+'reviews').head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## Задание №1:
Сколько книг вышло после 1 января 2000

In [12]:
query = '''SELECT COUNT(DISTINCT title)
           FROM books
           WHERE publication_date > '2000-01-01'
        '''

In [13]:
request(query)

,count
0,818


### Вывод:
В купленном сервисе **820 книг (вместо 782 книг без равно)**, эта информация позволяет понять какое количество книг должно быть в ьазе, чтобы заинтересовать новых пользователей.

## Задание №2:
Для каждой книги посчитайте количество обзоров и среднюю оценку

Добавила сортировку по средней оценке и количеству обзоров для себя

In [14]:
query = '''
           WITH
           book_rev AS (
                        SELECT book_id,
                               COUNT(reviews) AS count_rev
                        FROM reviews
                        GROUP BY book_id
                        )
                        
           SELECT DISTINCT b.title AS t,
                  MAX(brv.count_rev),
                  AVG(r.rating)
           FROM books AS b
           LEFT JOIN ratings AS r ON b.book_id = r.book_id
           LEFT JOIN book_rev AS brv ON b.book_id = brv.book_id
           GROUP BY t
           ORDER BY AVG(r.rating) DESC,
                    MAX(brv.count_rev) DESC
        '''

In [15]:
request(query).head(10)

,t,max,avg
0,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.0
1,A Dirty Job (Grim Reaper #1),4.0,5.0
2,Moneyball: The Art of Winning an Unfair Game,3.0,5.0
3,School's Out—Forever (Maximum Ride #2),3.0,5.0
4,A Fistful of Charms (The Hollows #4),2.0,5.0
5,A Woman of Substance (Emma Harte Saga #1),2.0,5.0
6,Act of Treason (Mitch Rapp #9),2.0,5.0
7,Alas Babylon,2.0,5.0
8,Angels Fall,2.0,5.0
9,Arrows of the Queen (Heralds of Valdemar #1),2.0,5.0


### Вывод:
1. Самая высокая средняя оценка - 5, скорее всего в сервисе использовалась 5-ти бальная шкала.
2. Самое высокое число обзоров среди популярных книг - 4.
3. Этот запрос позволяет найти самые топовые книги, которые обязательно должны присутствовать в базе книг.

## Задание №3:
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [16]:
query = '''SELECT p.publisher
           FROM publishers AS p
           LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY COUNT(DISTINCT b.book_id) DESC
           LIMIT 1           
        '''

In [17]:
request(query)

,publisher
0,Penguin Books


### Вывод:
Этот запрос позволяет найти самые крупные издательства, к которым стоит присмотреться при формировании базы книг.

## Задание №4:
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [18]:
query = '''
           SELECT a.author,
                  AVG(r.rating)
           FROM authors AS a
           LEFT JOIN books AS b ON a.author_id = b.author_id
           LEFT JOIN ratings AS r ON b.book_id = r.book_id
           WHERE b.book_id IN (
                               SELECT book_id
                               FROM ratings
                               GROUP BY book_id
                               HAVING COUNT(rating_id) > 49
                               
                               )
           GROUP BY a.author
           ORDER BY AVG(r.rating) DESC
           LIMIT 1           
        '''

In [19]:
request(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


### Вывод:
Этот запрос позволяет найти самых популярных авторов, к которым стоит присмотреться при формировании базы книг.

## Задание №5:
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [20]:
query = '''WITH
           count_review AS (
                            SELECT username,
                                   COUNT(review_id) AS count_rv
                            FROM reviews
                            WHERE username IN (
                                                SELECT username
                                                FROM ratings
                                                GROUP BY username
                                                HAVING COUNT(rating_id) > 50
                                               )
                            GROUP BY username
                            )
           SELECT ROUND(AVG(count_rv))
           FROM count_review    
        '''

In [21]:
request(query)

,round
0,24.0


### Вывод:
1. В среднем активные пользователи оставляют 24 оценки.
2. Срез пользователей в подзапросе WHERE - это целевая аудитория сервиса, его можно использовать для более детального изучения предпочтений, изучив оценки каждого пользователя.